# Load the Datasets

In [ ]:
%pip install duckdb numpy pandas

In [17]:
file_path = "/workspaces/Delivery-Time-Crisis-Analysis/Datasets"

In [1]:
import duckdb

conn = duckdb.connect("main.db")

In [20]:
conn.execute(f"""CREATE OR REPLACE TABLE events 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_events.csv")
    """)

conn.execute("SELECT * FROM events LIMIT 3").fetch_df()

,event_id,zone_id,event_type,start_time,end_time
0,EVT00001,106,Rain,2024-01-04 17:00:00,2024-01-04 19:00:00
1,EVT00002,101,Rain,2024-01-04 17:00:00,2024-01-04 19:00:00
2,EVT00003,106,Traffic_Jam,2024-01-09 14:00:00,2024-01-09 15:30:00


In [21]:
conn.execute(f"""CREATE OR REPLACE TABLE orders 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_orders.csv")
    """)

conn.execute("SELECT * FROM orders LIMIT 3").fetch_df()

,order_id,zone_id,store_id,order_time,delivery_time_mins,status,cancellation_reason
0,ORD1000000,101,1,2024-01-01 00:47:00,9.40,Delivered,None
1,ORD1000001,101,1,2024-01-01 00:08:00,8.89,Delivered,None
2,ORD1000002,101,3,2024-01-01 00:47:00,9.52,Delivered,None


In [22]:
conn.execute(f"""CREATE OR REPLACE TABLE store_load 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_store_load.csv")
    """)

conn.execute("SELECT * FROM store_load LIMIT 3").fetch_df()

,store_id,timestamp,active_riders,orders_in_queue,avg_rider_wait_time
0,1,2024-01-01,11,0,1.01
1,2,2024-01-01,6,3,1.50
2,3,2024-01-01,6,3,0.74


# Exploring the Dataset

In [2]:
conn.execute("""
SELECT
DATE(order_time) AS date,
COUNT(*) as orders,
ROUND(AVG(delivery_time_mins), 2) as avg_delivery_mins,
ROUND(AVG(CASE WHEN status = 'Cancelled' THEN 1.0 ELSE 0.0 END) * 100, 2) as cancel_rate_pct
FROM orders
GROUP BY DATE(order_time)
""").fetch_df()

,date,orders,avg_delivery_mins,cancel_rate_pct
0,2024-01-01,3383,9.15,1.74
1,2024-01-02,3383,9.10,1.74
2,2024-01-03,3383,10.13,2.10
3,2024-01-04,3383,9.14,1.54
4,2024-01-05,3383,10.11,2.04
5,2024-01-06,4409,9.15,1.97
6,2024-01-07,4409,9.11,1.88
7,2024-01-08,3383,9.13,2.13
8,2024-01-09,3383,9.14,1.86
9,2024-01-10,3383,9.11,1.80


*Starting the 24th, we see a spike in the avg delivery time - which has sustained till the end of the month*

In [3]:
conn.execute("""
SELECT
CASE 
    WHEN order_time BETWEEN '2024-01-17' AND '2024-01-24' THEN 'Previous 7 Days'
    WHEN order_time BETWEEN '2024-01-24' AND '2024-01-31' THEN 'Last 7 Days'
    ELSE ''
END AS time_period,
COUNT(*) AS total_orders,
ROUND(AVG(delivery_time_mins), 2) as avg_delivery_time,
ROUND(AVG(CASE WHEN status = 'Cancelled' THEN 1.0 ELSE 0.0 END) * 100, 2) as cancellation_rate_pct,
COUNT(CASE WHEN cancellation_reason = 'Too long' THEN 1 END) as cancelled_too_long
FROM orders
WHERE order_time > '2024-01-17'
GROUP BY 1

""").fetch_df()

,time_period,total_orders,avg_delivery_time,cancellation_rate_pct,cancelled_too_long
0,Last 7 Days,25732,12.24,7.92,1856
1,Previous 7 Days,25734,9.25,1.99,161


*Cancellations increased massively in the last 7 days*

# Testing the Hypothesis

## Hypothesis 1 - Delays due to Rain

In [14]:
conn.execute("""
SELECT
    e.zone_id,
    e.event_type,
    DATE(e.start_time) AS date,
    COUNT(o.*) AS total_orders
FROM events e
JOIN orders o
ON o.zone_id = e.zone_id AND o.order_time >= e.start_time AND o.order_time <= end_time 
GROUP BY e.zone_id, e.event_type, DATE(e.start_time)
ORDER BY DATE(e.start_time)
""").fetch_df()

,zone_id,event_type,date,total_orders
0,106,Rain,2024-01-04,60
1,101,Rain,2024-01-04,120
2,106,Traffic_Jam,2024-01-09,21
3,104,Rain,2024-01-10,60
4,102,Traffic_Jam,2024-01-13,87
5,102,Traffic_Jam,2024-01-27,78
6,105,Traffic_Jam,2024-01-28,79


*External events like Rain and Traffic Jam are not limited to the crisis period of 24th - 30th, hence we can assume that the delays were not caused by rain or traffic jam*

## Hypothesis 2 - Lack of Riders

In [24]:
conn.execute("""
        WITH orders_last_14_days AS (
        SELECT
             store_id,
             order_time,
             status,
             cancellation_reason,
             CASE 
                WHEN order_time BETWEEN '2024-01-17' AND '2024-01-24' THEN 'Previous 7 Days'
                WHEN order_time BETWEEN '2024-01-24' AND '2024-01-31' THEN 'Last 7 Days'
             END AS time_period,
             delivery_time_mins
        FROM orders
        WHERE order_time > '2024-01-17'
        )

        SELECT
             o.time_period,
             COUNT(o.*) AS total_orders,
             SUM(s.active_riders) AS active_riders,
             AVG(s.avg_rider_wait_time) AS avg_rider_wait_time,
             ROUND(AVG(o.delivery_time_mins), 2) as avg_delivery_time,
             ROUND(AVG(CASE WHEN o.status = 'Cancelled' THEN 1.0 ELSE 0.0 END) * 100, 2) as cancellation_rate_pct,
             COUNT(CASE WHEN o.cancellation_reason = 'Too long' THEN 1 END) as cancelled_too_long
        FROM store_load s
        JOIN orders_last_14_days o
        ON s.store_id = o.store_id AND s.timestamp >= o.order_time AND s.timestamp <= o.order_time
             GROUP BY o.time_period
""").fetch_df()

,time_period,total_orders,active_riders,avg_rider_wait_time,avg_delivery_time,cancellation_rate_pct,cancelled_too_long
0,Last 7 Days,1666,15692.0,3.135288,12.16,7.68,117
1,Previous 7 Days,1722,16423.0,1.017857,9.24,2.56,10
